In [2]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langserve import RemoteRunnable
import json
import bs4
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.vectorstores import VectorStoreRetriever
from langchain_core.documents.base import Document
from langchain.tools.retriever import create_retriever_tool
import os

In [3]:
from dotenv import load_dotenv

# 환경변수 로드 (.env)
load_dotenv()

True

In [4]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

# llm = OllamaFunctions(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", format="json", temperature=0)
llm = ChatOllama(model="EEVE-Korean-Instruct-10.8B-v1.0:latest", temperature=0)

In [5]:
embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
    encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
)

# 로컬 DB 불러오기
MY_NEWS_INDEX = "MY_NEWS_INDEX"
vectorstore1 = FAISS.load_local(MY_NEWS_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever1 = vectorstore1.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 유사도 높은 3문장 추출
MY_PDF_INDEX = "MY_PDF_INDEX"
vectorstore2 = FAISS.load_local(MY_PDF_INDEX, 
                               embeddings, 
                               allow_dangerous_deserialization=True)
retriever2 = vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k": 3}) # 유사도 높은 3문장 추출

/Users/user/Library/Caches/pypoetry/virtualenvs/test-langchain-lw2NDlv9-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/user/Library/Caches/pypoetry/virtualenvs/test-langchain-lw2NDlv9-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.tools.retriever import create_retriever_tool
from langchain_core.pydantic_v1 import BaseModel, Field

retriever_tool1 = create_retriever_tool(
    retriever1,
    name="saved_news_search",
    description="엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다. 엔비디아, 퍼플렉시티, 라마3 관련 정보는 이 도구를 사용해야 한다",
)

retriever_tool2 = create_retriever_tool(
    retriever2,
    name="pdf_search",
    description="생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다. 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보는 이 도구를 사용해야 한다",
)

tools = [retriever_tool1, retriever_tool2]
tools

[Tool(name='web_search', description='엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다. 엔비디아, 퍼플렉시티, 라마3 관련 정보는 이 도구를 사용해야 한다', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10f5fd940>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x294389890>, search_kwargs={'k': 3}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x10f5fdb20>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x294389890>, search_kwargs={'k': 3}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')),
 Tool(name='pdf_search', description='생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다. 

In [19]:
prompt_for_select_tool = ChatPromptTemplate.from_messages([
    ("system", """
You have "tools" that can answer "question".
Using "tools" as a guide, choose a "tool" that can answer "question".
Without saying anything else, say the "tool_name" of the selected "tool" in English.
If there is no appropriate 'tool', say 'None'.

<tools>
{tools}
</tools>

<question>
{question}
</question>

# answer :
"""
    )
])

def get_tools(query):
    tool_info = [{"tool_name": tool.name, "tool_description": tool.description} for tool in tools]
    print(f"get_tools / {tool_info}")
    return json.dumps(tool_info, ensure_ascii=False)

chain_for_select_tool = (
    {"tools": get_tools, "question": RunnablePassthrough()}
    | prompt_for_select_tool 
    | llm
    | StrOutputParser()
    )

In [20]:
query = "라마3 성능은?"
# query = "생성형 AI 도입에 따른 규제 연구 책임자는?"
selected_tool = chain_for_select_tool.invoke(query)
selected_tool

get_tools / [{'tool_name': 'web_search', 'tool_description': '엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다. 엔비디아, 퍼플렉시티, 라마3 관련 정보는 이 도구를 사용해야 한다'}, {'tool_name': 'pdf_search', 'tool_description': '생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다. 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보는 이 도구를 사용해야 한다'}]


'web_search'

In [8]:
def get_retriever_by_tool_name(name) -> VectorStoreRetriever:
    print(f"get_retriever_by_tool_name / name: {name}")
    for tool in tools:
        if tool.name == name:
            # print(tool.func) # functools.partial(<function _get_relevant_documents at 0x1487dd6c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x317e52ea0>, search_kwargs={'k': 5}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n')
            return tool.func.keywords['retriever']
    return None

retriever = get_retriever_by_tool_name(selected_tool)
retriever.invoke(query)

get_retriever_by_tool_name / name: pdf_search


[Document(page_content='15\n나. 생성형 AI 규제 유형화\n○본 연구에서는 주요국의 생성형 AI 규제 동향을 ‘AI 관련 입법 목적’ \n및 ‘AI에 대한 규제는 규제원칙 및 방식’에 따라 아래와 같이 분류한다.\n○다수 국가들이 AI 규제를 준비하고 있는 가운데, 미국과 EU는 AI에 대\n한 규제 입법에 앞장서 왔다. 미국과 EU의 규제 내용 및 동향 대한 자\n세한 내용은 4장에서 다룰 것이다.\n- 2019년 트럼프 전 대통령이 행정명령 제13859호에서 “미국 인공지\n능 구상”을 공표한 이후, 미국은 AI 관련 규제를 입법하였다. 본 연\n구에서는 AI 연구개발과 훈련과 관련한 법률인 「생성적 적대 신경\n망 출력물 확인법」, 「국가 인공지능 구상법 2020」을 분석한다. 또\n한 연방정부가 제정한 AI 사용에 관한 행정명령 제13960호, 「정부인\n공지능법 2020」, 「조달인력의 인공지능 역량강화법」, 「미국인공\n지능진흥법」 등과 더불어 최근 논의되고 있는 생성형 AI 기술 규제', metadata={'source': '/Users/user/kykdev/7000_AI/test_langchain/assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'file_path': '/Users/user/kykdev/7000_AI/test_langchain/assets/생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf', 'page': 20, 'total_pages': 227, 'format': 'PDF 1.4', 'title': '', 'author': 'fpqlt', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.9139', 'producer': 'Hancom PDF 1.3.0.538', 'creationDate': "D:20231228104945+09'00'", 'modDate': "D:20231228104945+09'00'

In [9]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
너는 유능한 업무 보조자야.
다음 context를 사용해서 question에 대한 답을 말해줘.
정답을 모르면 모른다고만 해. 한국어로 대답해.

<question>
{question}
</question>

<context>
{context}
</context>

# answer :
"""
    ),
])

retrieved_docs = []
def get_page_contents_with_metadata(docs) -> str: 
    global retrieved_docs
    retrieved_docs = docs
    
    result = ""
    for i, doc in enumerate(docs):
        if i > 0:
            result += "\n\n"
        result += f"## 본문: {doc.page_content}\n### 출처: {doc.metadata['source']}"
    return result

def get_retrieved_docs(query) -> str:
    selected_tool = chain_for_select_tool.invoke(query)
    retriever = get_retriever_by_tool_name(selected_tool)
    docs = retriever.invoke(query)
    return get_page_contents_with_metadata(docs)

def get_metadata_sources(docs) -> str: 
    sources = set()
    for doc in docs:
        source = doc.metadata['source']
        is_pdf = source.endswith('.pdf')
        if (is_pdf):
            file_path = doc.metadata['source']
            file_name = os.path.basename(file_path)
            source = f"{file_name} ({int(doc.metadata['page']) + 1}페이지)"
        sources.add(source)
    return "\n".join(sources)

def parse(ai_message: AIMessage) -> str:
    """Parse the AI message and add source."""
    return f"{ai_message.content}\n\n[출처]\n{get_metadata_sources(retrieved_docs)}"

agent_chain = (
    {"context": get_retrieved_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | parse
)

# agent_chain.invoke("퍼플렉시티가 투자받은 금액?") # saved_news_search / '퍼플렉시티는 투자받은 금액이 약 6300만 달러(약 860억 원)이며, 회사 가치는 10억 달러(약 1조 3760억 원) 이상으로 평가받았습니다.'
agent_chain.invoke("생성형 AI 도입에 따른 규제 연구 책임자는?") # pdf_search / '해당 문서에 따르면, 생성형 AI 도입과 관련된 규제 연구를 담당하는 연구 책임자는 김주희 교수님입니다.'

get_tools / [('web_search', '엔비디아, 퍼플렉시티, 라마3 관련 정보를 검색한다. 엔비디아, 퍼플렉시티, 라마3 관련 정보는 이 도구를 사용해야 한다'), ('pdf_search', '생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보를 검색한다. 생성형 AI 신기술 도입에 따른 선거 규제 연구 관련 정보는 이 도구를 사용해야 한다')]
get_retriever_by_tool_name / name: pdf_search


'생성형 AI 도입에 따른 규제 연구 책임자는 김주희 교수님입니다.\n\n[출처]\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (1페이지)\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (6페이지)\n생성형_AI_신기술_도입에_따른_선거_규제_연구_결과보고서.pdf (20페이지)'